In [12]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
import pandas as pd
from molar import ClientConfig, Client
import rdkit
import rdkit.Chem as chem
import rdkit.Chem.AllChem as allchem

from molar_interface import MolarInterface

In [14]:
# define user details
user_details = {
    'email':'riley.hickman13@gmail.com',
    'password':'az1$jH5v',
}


In [17]:
mol = MolarInterface(user_details, database_name='madness_laser_test') # client in client attribute


In [19]:
mol.client.test_token()

{'email': 'riley.hickman13@gmail.com',
 'is_superuser': False,
 'is_active': True,
 'full_name': 'rileyhickman',
 'created_on': '2022-01-18T19:16:33.241173',
 'user_id': 3}

## MolarInterface functionality

### Querying data from DB

'AVAILABLE', 'ACQUIRED',  'PROCESSING',  'SYNTHESIZED',  'SHIPPED',  'RECEIVED',  'DONE', 'FAILED'

* get_synthesis_details(identifier)  --> get details about a specifc synthesis from the DB
* get_syntheses(lab, instrument, status) --> details about all the syntheses set to a particular status (see above)
* get_target_molecule(identifier, identifier_type='hid') --> queries database for a specific target molecule
* get_fragment_details(identifier, identifier_type='hid') --> get details about specific fragment (ID, SMILES, CAS)


### Writing data to the DB

* create_target_compund(smiles, fragment_a, fragment_b, fragment_c, fragment_identifier='hid') --> creates all entries for the synthesis of a new compound (molecule, molecule_molecule, synthesis)
* update_synthesis_status(identifier, status, identifier_type='hid') --> updates the status of a synthesis
* update_optics_data(identifier, optics_data, identifier_type='hid')



In [33]:
# get the syntheses
df = mol.get_syntheses(lab='Toronto', instrument='ChemSpeed', status='available')
df.head()

,synthesis.synthesis_id,synthesis.status,synthesis.molecule_id,lab.name,machine.machine_id,product.smiles,fragment_a.hid,fragment_a.smiles,fragment_b.hid,fragment_b.smiles,fragment_c.hid,fragment_c.smiles
0,292f21b5-1953-4762-94cb-4722f8c720bc,available,5acc0682-2fd8-4934-9e58-881bd5b038ec,Toronto,f158c26e-a94e-4994-9a95-de324aa1da23,CCCCCCCCCCCCCC,A027,Cc1ccc2c(c1)c1cc(C)ccc1n2-c1ccc(B2OC(C)(C)C(C)...,B002,C[N+]12CC(=O)O[B-]1(c1sccc1Br)OC(=O)C2,C121,Cc1nc(NN)c(Br)cc1Br
1,230b32ce-acd4-46a8-859d-d0b71e15f039,available,0b0b9fdf-df94-4316-ae89-5628601d6dcc,Toronto,f158c26e-a94e-4994-9a95-de324aa1da23,CCCCCCCCCCCCCCCCCCC,A027,Cc1ccc2c(c1)c1cc(C)ccc1n2-c1ccc(B2OC(C)(C)C(C)...,B002,C[N+]12CC(=O)O[B-]1(c1sccc1Br)OC(=O)C2,C121,Cc1nc(NN)c(Br)cc1Br
2,919e10b2-261e-4604-b494-20e560ab517b,available,97e4b1fb-a1c1-40b9-a4dc-c5715bba40e6,Toronto,f158c26e-a94e-4994-9a95-de324aa1da23,CCCCCCCCCCCCCCCCC,A027,Cc1ccc2c(c1)c1cc(C)ccc1n2-c1ccc(B2OC(C)(C)C(C)...,B002,C[N+]12CC(=O)O[B-]1(c1sccc1Br)OC(=O)C2,C121,Cc1nc(NN)c(Br)cc1Br
3,22ec7d60-ba7d-4147-9a04-df564714f1f8,available,b3e77fd2-683b-450a-913c-97998dcff2e0,Toronto,f158c26e-a94e-4994-9a95-de324aa1da23,CCCCCCCCCCCCCCCCCC,A027,Cc1ccc2c(c1)c1cc(C)ccc1n2-c1ccc(B2OC(C)(C)C(C)...,B002,C[N+]12CC(=O)O[B-]1(c1sccc1Br)OC(=O)C2,C121,Cc1nc(NN)c(Br)cc1Br
4,70d5265a-76e5-41ce-9a5d-0bd7b4cec93f,available,29d07431-c3fd-4538-a2a1-2f7362e76068,Toronto,f158c26e-a94e-4994-9a95-de324aa1da23,CCCCCCCCCCCCCCCCCCCCCCCCC,A027,Cc1ccc2c(c1)c1cc(C)ccc1n2-c1ccc(B2OC(C)(C)C(C)...,B002,C[N+]12CC(=O)O[B-]1(c1sccc1Br)OC(=O)C2,C121,Cc1nc(NN)c(Br)cc1Br


In [35]:
# get synthesis details
df = mol.get_synthesis_details(identifier='292f21b5-1953-4762-94cb-4722f8c720bc')
df.head()

,synthesis.synthesis_id,synthesis.status,synthesis.molecule_id,fragment_a.hid,fragment_a.smiles,fragment_b.hid,fragment_b.smiles,fragment_c.hid,fragment_c.smiles
0,292f21b5-1953-4762-94cb-4722f8c720bc,available,5acc0682-2fd8-4934-9e58-881bd5b038ec,A027,Cc1ccc2c(c1)c1cc(C)ccc1n2-c1ccc(B2OC(C)(C)C(C)...,B002,C[N+]12CC(=O)O[B-]1(c1sccc1Br)OC(=O)C2,C121,Cc1nc(NN)c(Br)cc1Br


In [44]:
# update synthesis status
mol.update_synthesis_status(identifier='A027B002C121',
                            status='DONE',
                            identifier_type='hid',
                        )

KeyError: 'molecule_id'

In [40]:
df = mol.get_synthesis_details(identifier='292f21b5-1953-4762-94cb-4722f8c720bc')
df.head()

,synthesis.synthesis_id,synthesis.status,synthesis.molecule_id,fragment_a.hid,fragment_a.smiles,fragment_b.hid,fragment_b.smiles,fragment_c.hid,fragment_c.smiles
0,292f21b5-1953-4762-94cb-4722f8c720bc,available,5acc0682-2fd8-4934-9e58-881bd5b038ec,A027,Cc1ccc2c(c1)c1cc(C)ccc1n2-c1ccc(B2OC(C)(C)C(C)...,B002,C[N+]12CC(=O)O[B-]1(c1sccc1Br)OC(=O)C2,C121,Cc1nc(NN)c(Br)cc1Br


In [46]:
mol.get_target_molecule(identifier='5acc0682-2fd8-4934-9e58-881bd5b038ec', identifier_type='molecule_id')

,molecule_id,created_on,updated_on,smiles,molecule_type_id
0,5acc0682-2fd8-4934-9e58-881bd5b038ec,2022-02-02T16:53:52.339694,2022-02-02T16:53:52.339694,CCCCCCCCCCCCCC,a06a38b6-78b6-4fcc-8a61-3fe9d0af7cea


In [48]:
mols = mol.client.query_database(
        types='molecule_molecule',
)
mols

,molecule_molecule_id,created_on,updated_on,fragment_a,fragment_b,fragment_c,molecule_id
0,288904db-93ed-4cd9-a408-20fd562beca6,2022-02-01T22:33:36.442040,2022-02-01T22:33:36.442040,c48d4657-acde-42f1-86ed-ad85bf0f3b5f,5e02fe87-04eb-4d8e-b2d4-78f2ae025546,5b92973e-a494-429f-a7a6-ae45a421bcfb,db38dc00-a9b3-4c65-9632-2f17ff16568d
1,1661a343-a837-4438-986b-e026a25755f3,2022-02-02T16:53:52.467010,2022-02-02T16:53:52.467010,faa4de17-7959-4bc4-a071-de1fda1b8a05,80ae3faf-ebe1-4306-ab0b-7546691d09ab,e10ae2cd-e8ad-499f-a9cb-6a0ca1b92a1c,5acc0682-2fd8-4934-9e58-881bd5b038ec
2,41bbfc39-7356-473f-91b5-bb0bc7a88af8,2022-02-02T17:01:39.479074,2022-02-02T17:01:39.479074,faa4de17-7959-4bc4-a071-de1fda1b8a05,80ae3faf-ebe1-4306-ab0b-7546691d09ab,e10ae2cd-e8ad-499f-a9cb-6a0ca1b92a1c,0b0b9fdf-df94-4316-ae89-5628601d6dcc
3,4389d2e7-922a-48ce-bbd2-78e4937c4f7c,2022-02-02T17:08:05.296666,2022-02-02T17:08:05.296666,faa4de17-7959-4bc4-a071-de1fda1b8a05,80ae3faf-ebe1-4306-ab0b-7546691d09ab,e10ae2cd-e8ad-499f-a9cb-6a0ca1b92a1c,b0711df4-5e11-42dc-b136-b87060cfa5ea
4,216eade7-7cd0-4bd6-b16b-9e8226162857,2022-02-02T17:13:39.764532,2022-02-02T17:13:39.764532,faa4de17-7959-4bc4-a071-de1fda1b8a05,80ae3faf-ebe1-4306-ab0b-7546691d09ab,e10ae2cd-e8ad-499f-a9cb-6a0ca1b92a1c,97e4b1fb-a1c1-40b9-a4dc-c5715bba40e6
5,73ab3618-0c19-49bf-96de-4fb32e1d0577,2022-02-02T17:14:26.419078,2022-02-02T17:14:26.419078,faa4de17-7959-4bc4-a071-de1fda1b8a05,80ae3faf-ebe1-4306-ab0b-7546691d09ab,e10ae2cd-e8ad-499f-a9cb-6a0ca1b92a1c,b3e77fd2-683b-450a-913c-97998dcff2e0
6,1633492d-be3a-48ae-bf42-09a036a972c8,2022-02-02T17:50:00.596877,2022-02-02T17:50:00.596877,faa4de17-7959-4bc4-a071-de1fda1b8a05,80ae3faf-ebe1-4306-ab0b-7546691d09ab,e10ae2cd-e8ad-499f-a9cb-6a0ca1b92a1c,29d07431-c3fd-4538-a2a1-2f7362e76068
7,36b60134-1da0-4991-b6b2-b436cbfd385e,2022-02-02T17:57:17.583248,2022-02-02T17:57:17.583248,faa4de17-7959-4bc4-a071-de1fda1b8a05,80ae3faf-ebe1-4306-ab0b-7546691d09ab,e10ae2cd-e8ad-499f-a9cb-6a0ca1b92a1c,568d6ebf-a2f6-4652-bcda-93d6c6a7133f
8,48a16b28-5912-489c-93ce-8f40a737b9b8,2022-02-02T17:59:41.474611,2022-02-02T17:59:41.474611,faa4de17-7959-4bc4-a071-de1fda1b8a05,80ae3faf-ebe1-4306-ab0b-7546691d09ab,e10ae2cd-e8ad-499f-a9cb-6a0ca1b92a1c,7e3457e5-3859-4be8-8159-d3ef25caf21f
9,5515bca7-c8ca-4eb2-9c5a-3c1d09a22de4,2022-02-02T18:30:39.540706,2022-02-02T18:30:39.540706,faa4de17-7959-4bc4-a071-de1fda1b8a05,80ae3faf-ebe1-4306-ab0b-7546691d09ab,e10ae2cd-e8ad-499f-a9cb-6a0ca1b92a1c,2985090b-db2a-4d74-aa04-8307940d87f6
